In [102]:
import os
from pathlib import Path
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.restoration import denoise_wavelet
from skimage.feature import local_binary_pattern
from sklearn import neighbors, linear_model
from sklearn.neural_network import MLPClassifier
import cv2

In [103]:
# Feature Extration - Local Binary Pattern
def lbp_feature_extraction(im, n_points, radius, method):
    im_gray = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2GRAY)
    lbp = local_binary_pattern(im_gray, n_points, radius, method)
    # Features returned are the hisogram of resulting data from lbp algorithm
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, radius + 2), normed=True)
    return hist

In [104]:
# Feature Extraction - Denoise Wavelets
def den_wavelets(im):
    im_clean = denoise_wavelet(im)
    noise = im - im_clean
    return noise

In [105]:
def logistic_reg(data, target):
    log = logistic.fit(data, target)
    return log

In [106]:
def neural_network():
    pass

In [107]:
def k_nearest_neighbor():
    pass

In [108]:
is_kaggle = True

# Generating train and test path
if is_kaggle:
    input_path = Path('../input')
else:
    input_path = Path('../input')
train_path = input_path / 'train'
test_path = input_path / 'test'

In [109]:
# Listing all directories in trainning path
cameras = os.listdir(train_path)

In [110]:
# Defining size of trainning
size_train = 0.8

In [116]:
# Generating DataFrame of images and labes in the trainning and validation dataset
train_images = []
n_pictures = []

count = 0

for camera in cameras:
    pic = len(os.listdir(train_path / camera))
    n_pictures.append(pic)
    for fname in sorted(os.listdir(train_path / camera)):
            train_images.append((camera, fname))
train = pd.DataFrame(train_images, columns=['camera', 'fname'])

In [117]:
# Generating DataFrame of images and labes in the testing dataset
test_images = []
for fname in sorted(os.listdir(test_path)):
    test_images.append(fname)
test = pd.DataFrame(test_images, columns=['fname'])

In [122]:
'''# Procedure to mount feature vectors
train_lbp = []
train_wav = []
valid_lbp = []
valid_wav = []
train_target = []
valid_target = []

j = 0

start = time.time()
for camera in cameras:
    print("Feature extraction: %s"%(camera))
    for i in range(n_pictures[j]):
        print("Example %i"%i, end = '\r')
        seg = train[train['camera'] == camera]
        seg = seg.reset_index()
        path_im = train_path / seg.at[i, 'camera'] / seg.at[i, 'fname']
        bgr_image = cv2.imread(str(path_im))
        rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
        im = cv2.resize(rgb_image, (32, 32))
        if i < int(size_train*n_pictures[j]):
            train_wav.append(den_wavelets(im))
            train_lbp.append(lbp_feature_extraction(im, 24, 8, 'uniform'))
            train_target.append(seg.at[0, 'camera'])
        else:
            valid_wav.append(den_wavelets(im))
            valid_lbp.append(lbp_feature_extraction(im, 24, 8, 'uniform'))
            valid_target.append(seg.at[0, 'camera'])
    j += 1
    print("Extraction finished!\n")
end = time.time()

print('The time elapsed to extract all lbp features was: %.2f min'%((end-start)/60))'''

Feature extraction: Samsung-Galaxy-Note3
Extraction finished!

Feature extraction: Sony-NEX-7
Extraction finished!

Feature extraction: Motorola-Droid-Maxx
Extraction finished!

Feature extraction: iPhone-4s
Extraction finished!

Feature extraction: HTC-1-M7
Extraction finished!

Feature extraction: Motorola-X
Extraction finished!

Feature extraction: iPhone-6
Extraction finished!

Feature extraction: Motorola-Nexus-6
Extraction finished!

Feature extraction: LG-Nexus-5x
Extraction finished!

Feature extraction: Samsung-Galaxy-S4
Extraction finished!

The time elapsed to extract all lbp features was: 596.712909


In [157]:
train_wav = np.array(train_wav)
train_wav = train_wav.reshape(np.shape(train_wav)[0],-1)

valid_wav = np.array(valid_wav)
valid_wav = valid_wav.reshape(np.shape(valid_wav)[0],-1)


#Predicting using logistic regression
logistic = linear_model.LogisticRegression()
print('Logistic Regression')
print('LBP - Score: %.2f'%(logistic.fit(train_lbp, train_target).score(valid_lbp, valid_target)))
print('WAV - Score: %.2f'%(logistic.fit(train_wav, train_target).score(valid_wav, valid_target)))

# Predicting using logistic regression
knn = neighbors.KNeighborsClassifier()
print("(KNN) Score: %.2f"%(knn.fit(train_lbp, train_target).score(valid_lbp, valid_target)))

# Predicting using logistic regression
mlp = MLPClassifier(alpha=1)
print("(MLP) Score: %.2f"%(mlp.fit(train_lbp, train_target).score(valid_lbp, valid_target)))

Logistic Regression
LBP - Score: 0.14
WAV - Score: 0.20
(KNN) Score: 0.13
(MLP) Score: 0.12


In [ ]:
# Tests
seg = train[train['camera'] == cameras[1]]
seg = seg.reset_index()
path_im = train_path / seg.get_value(0, 'camera') / seg.get_value(120, 'fname')
bgr_image = cv2.imread(str(path_im))
rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
im = cv2.resize(rgb_image, (128, 128))
#im = rgb_image
im_gray = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2GRAY)
plt.imshow(im)
plt.figure()
#plt.imshow(rgb_image)

#plt.imshow(im)
noise = den_wavelets(im)
plt.figure()
plt.imshow(np.round(noise))
(fig, ax) = plt.subplots()
print(np.shape(noise))
print(np.shape(noise.ravel()))
ax.hist(np.round(noise).ravel(), normed=True, bins=20, range=(0,256))